# pretrained된 모델로 prediction만 수행

In [1]:
import torch
from torch import nn, Tensor
from torch.optim import Optimizer
from torch.utils.data import DataLoader, RandomSampler, DistributedSampler, random_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import CrossEntropyLoss
from pytorch_lightning.core.lightning import LightningModule 
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.metrics.functional import accuracy, precision, recall
from transformers import AdamW, BertForSequenceClassification, AdamW, BertConfig, AutoTokenizer, BertTokenizer, TrainingArguments
from keras.preprocessing.sequence import pad_sequences


import random
import numpy as np 

import time
import datetime
import pandas as pd
import os
from tqdm import tqdm


import pandas as pd
from transformers import AutoTokenizer, AutoModelWithLMHead
from keras.preprocessing.sequence import pad_sequences


In [2]:
if torch.cuda.is_available():    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070


In [3]:
test_set = pd.read_csv('/home/aiffel-dj57/project/labeled_water_barrel.csv')
test_set

,Unnamed: 0,name,ratings,date,comment,search,keyword,label
0,0,김성주,5,9시간 전,가편하게 산책과 등산을 할 수 있는 곳,계룡산국립공원수통골지구,water_barrel,1
1,1,서알콩,5,1일 전,커피 한잔하기 딱 좋은 거리에 용,계룡산국립공원수통골지구,water_barrel,1
2,2,윤설희,5,2일 전,비 오고 난 수통골,계룡산국립공원수통골지구,water_barrel,1
3,3,이상훈,4,2일 전,가족 나들이하기 정말 좋은 곳 국립이라 관리도 잘 돼있네요,계룡산국립공원수통골지구,water_barrel,1
4,4,송순이공주,5,2일 전,동네라서 자주 가는데 갈 때마다 좋은 곳이에요 이에는 뱀이 나와서 좀 놀랐어요,계룡산국립공원수통골지구,water_barrel,1
...,...,...,...,...,...,...,...,...
680,680,김태규,4,4년 전,완만한 산책 등산 먹거리 좋아요,수통골유원지,water_barrel,1
681,681,김뚱이,5,4년 전,가깝고 깨끗하게 정리도 잘 되어 있어서 좋아요,수통골유원지,water_barrel,1
682,682,이시형,5,4년 전,가끔 산책하러 가는 수통골이에요,수통골유원지,water_barrel,1
683,683,Park Sangbae,4,4년 전,등산을 목적으로 한다면 다녀올만함 여러 코스가 있으니 취향과 체력에 맞춰서 다녀오시...,수통골유원지,water_barrel,-1


In [4]:
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

In [5]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "beomi/kcbert-base",
    num_labels = 2, 
                    
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)


Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [6]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30000, 768)
bert.embeddings.position_embeddings.weight                (300, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [7]:
def convert_input_data(sentences):

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 64

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
        
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [8]:
def test_sentences(sentences):
 
    # 평가모드로 변경!!!!!
    model.eval()

    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [9]:
device = "cuda:0"
model = model.to(device)

In [10]:
logits = test_sentences(['기대를 하지 않고 가면 충분히 즐길만합니다.'])

print(logits)
print(np.argmax(logits)) #긍

[[0.09950283 0.21006171]]
1


In [11]:
logits = test_sentences(['야간 개장을 시작하여 선선하게 둘러볼 수 있어 더 좋은 것 같다.'])

print(logits)
print(np.argmax(logits)) #긍

[[-0.16729078  0.17005226]]
1


In [12]:
logits = test_sentences(['놀이기구 운영을 너무 천천히 해요 기구 타려면 너무 오래 기다려야 해요.'])

print(logits)
print(np.argmax(logits)) #부

[[0.18517739 0.16900083]]
0


In [13]:
logits = test_sentences(['아이들의 놀이터 하지만 어른들의 새치기 주말은 피하시길다.'])

print(logits)
print(np.argmax(logits)) #긍

[[-0.01652993  0.40031958]]
1


In [14]:
logits = test_sentences(['좋긴 한데 사람이 너무 많아서'])

print(logits)
print(np.argmax(logits)) #부

[[ 0.18033575 -0.1233479 ]]
0


In [15]:
logits = test_sentences(['동네라서 자주 가는데 갈 때마다 좋은 곳이에요 이에는 뱀이 나와서 좀 놀랐어요'])

print(logits)
print(np.argmax(logits)) #긍

[[-0.34756047 -0.18381375]]
1


In [16]:
logits = test_sentences(['간단하게 발족 욕을 할 수 있는 족욕 쟁입니다 일부러 들릴만한 곳은 아니지만 쉬어가기에는 좋은 것 같네요 물을 자주 갈아주는 것 같지는 않네요'])

print(logits)
print(np.argmax(logits)) #긍

[[0.01437217 0.332919  ]]
1


In [17]:
logits = test_sentences(['현재 코로나 때에 물다 빠져 있고 체험 불가'])

print(logits)
print(np.argmax(logits)) # 긍

[[-0.1033605   0.02618557]]
1


In [18]:
logits = test_sentences(['유성온천 근처의 산책과 함께 족욕을 즐길 수 있는 곳 사람이 많을 때는 물이 약간 깨끗하지 않음'])

print(logits)
print(np.argmax(logits)) #긍

[[-0.19290826  0.08791158]]
1


In [19]:
logits = test_sentences(['산책로가 잘되어 있어 산책하기 좋습니다 등산로도 있는데 등산하신다면 신발과 배낭 그리고 물은 챙기시는 게 좋을 거 같습니다 등산로 지형은 간혹 가파른 곳이 있어 안전사고에 주의가 필요합니다'])

print(logits)
print(np.argmax(logits)) #긍

[[-0.09999039  0.14928955]]
1


# 체크 

1. label num = 3 을 넣고 수행해봤더니 0으로만 계산되어 나왔다.
2. 긍정-부정이 같이 있으면 0(부정)으로 확인함을 알 수 있다

# 영화 데이터로 파인튜닝한 모델을 불러와 prediction 수행

In [20]:
# Load the checkpoint
model2 = BertForSequenceClassification.from_pretrained("./sentiment-analysis/checkpoint-3500", num_labels = 2, 
                    
    output_attentions = False,
    output_hidden_states = False, 
)

In [21]:
def test_sentences(sentences):
 
    # 평가모드로 변경!!!!!
    model2.eval()

    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model2(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [22]:
device = "cuda:0"
model = model2.to(device)

In [23]:
logits = test_sentences(['기대를 하지 않고 가면 충분히 즐길만합니다.'])

print(logits)
print(np.argmax(logits))  #긍

[[0.20660742 0.5705789 ]]
1


In [24]:
logits = test_sentences(['야간 개장을 시작하여 선선하게 둘러볼 수 있어 더 좋은 것 같다.'])

print(logits)
print(np.argmax(logits)) #긍

[[0.4956115 0.7484025]]
1


In [25]:
logits = test_sentences(['놀이기구 운영을 너무 천천히 해요 기구 타려면 너무 오래 기다려야 해요.'])

print(logits)
print(np.argmax(logits)) #부

[[0.629299  0.2787813]]
0


In [26]:
logits = test_sentences(['아이들의 놀이터 하지만 어른들의 새치기 주말은 피하시길다.'])

print(logits)
print(np.argmax(logits)) #부

[[0.6168568  0.02267386]]
0


In [27]:
logits = test_sentences(['좋긴 한데 사람이 너무 많아서'])

print(logits)
print(np.argmax(logits)) #부

[[0.653729   0.47054613]]
0


In [28]:
logits = test_sentences(['동네라서 자주 가는데 갈 때마다 좋은 곳이에요 이에는 뱀이 나와서 좀 놀랐어요'])

print(logits)
print(np.argmax(logits)) #긍

[[0.012468   0.36440152]]
1


In [29]:
logits = test_sentences(['간단하게 발족 욕을 할 수 있는 족욕 쟁입니다 일부러 들릴만한 곳은 아니지만 쉬어가기에는 좋은 것 같네요 물을 자주 갈아주는 것 같지는 않네요'])

print(logits)
print(np.argmax(logits)) #긍

[[0.30119035 0.7184674 ]]
1


In [30]:
logits = test_sentences(['현재 코로나 때에 물다 빠져 있고 체험 불가'])

print(logits)
print(np.argmax(logits)) #부

[[0.4763077  0.30161127]]
0


In [31]:
logits = test_sentences(['유성온천 근처의 산책과 함께 족욕을 즐길 수 있는 곳 사람이 많을 때는 물이 약간 깨끗하지 않음'])

print(logits)
print(np.argmax(logits)) #긍

[[0.3433645  0.41356367]]
1


In [32]:
logits = test_sentences(['산책로가 잘되어 있어 산책하기 좋습니다 등산로도 있는데 등산하신다면 신발과 배낭 그리고 물은 챙기시는 게 좋을 거 같습니다 등산로 지형은 간혹 가파른 곳이 있어 안전사고에 주의가 필요합니다'])

print(logits)
print(np.argmax(logits)) #부

[[0.53869146 0.4992556 ]]
0
